#### 라이브러리 import

In [1]:
import pandas as pd
import numpy as np

from datasets import load_dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams
from sentence_transformers import SentenceTransformer

#### Method 정의
* retrieval에서, query에 대한 키워드 추출 및 query_filter에서 이를 사용하도록 하는 기능 추가!

In [2]:
def chunking(documents_df_, chunk_size_, chunk_overlap_):
    """
    chunk_size_, chunk_overlap_에 따라, documents_df_의 텍스트를 Chunking
    """
    chunk_results = []
    chunk_idx = 0
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size_,
        chunk_overlap=chunk_overlap_,
        length_function=len
    )

    for idx, row in documents_df_.iterrows():
        origin_text = row['summary']

        chunks = text_splitter.create_documents([origin_text])

        for chunk in chunks:
            chunk_results.append({
                'chunk_id':chunk_idx,
                'text':chunk.page_content,
                'metadata':{
                    'category':row['category'],
                    'press':row['press'],
                    'title':row['title'],
                    'chunk_size':len(chunk.page_content)
                }
            })
            chunk_idx += 1

    return chunk_results

def embedding_indexing(chunk_results_, embedding_model_name_, collection_name_, use_instruct_prefix=False):
    # Chunking
    chunk_results = chunk_results_

    # Retriever 모델 선택 및 임베딩
    embedding_model = SentenceTransformer(embedding_model_name_)

    EMBEDDING_DIM = embedding_model.get_sentence_embedding_dimension() # 임베딩 차원 확인

    chunk_texts = [chunk['text'] for chunk in chunk_results] # 청크 텍스트 리스트

    if use_instruct_prefix:
        chunk_texts = [f"passage: {chunk_text}" for chunk_text in chunk_texts]

    embeddings = embedding_model.encode(chunk_texts, show_progress_bar=True) # 임베딩 / 임베딩 과정 시각화

    # Qdrant 컬렉션 구축
    client = QdrantClient(host='localhost', port=6333)

    client.recreate_collection(
        collection_name=collection_name_,
        vectors_config=models.VectorParams(
            size=EMBEDDING_DIM,
            distance=models.Distance.COSINE # 코사인 유사도
        )
    )

    # Indexing: Qdrant에 포인트 삽입
    points = []

    # Qdrant에 삽입할 PointStruct 리스트 생성
    for idx, chunk in enumerate(chunk_results):
        # metadata를 Payload로 사용, 원문 'text'를 포함
        payload_data = chunk['metadata']
        payload_data['text'] = chunk['text']

        # PointStruct 생성: id는 chunk_id(0부터 시작) 사용
        points.append(
            models.PointStruct(
                id=chunk['chunk_id'], # 청크 ID를 Qdrant의 고유 ID로 사용
                vector=embeddings[idx].tolist(), # numpy 배열을 list로 변환하여 삽입
                payload=payload_data
            )
        )

    # 데이터 삽입(upsert): Batch 단위로 삽입
    BATCH_SIZE = 1000

    for i in range(0, len(points), BATCH_SIZE):
        batch_points = points[i:i+BATCH_SIZE]

        try:
            operation_info = client.upsert(
                collection_name=collection_name_,
                wait=True,
                points=batch_points
            )

        except Exception as e:
            print(f"Error: 배치 {int(i/BATCH_SIZE)} 삽입 실패: {e}")
            
            return
        
    # 정상 작동 확인
    collection_info = client.count(
        collection_name=collection_name_,
        exact=True
    )

    if collection_info.count == len(points):
        print(f"임베딩 및 인덱싱 정상적으로 실행 완료 됨!")

def retrieval(embedding_model_name: str, collection_name: str, query: str, top_k: int, use_instruct_prefix=False):
    """
    Query를 임베딩하고, Qdrant 컬렉션에서 관련 문서를 검색.
    """

    # 임베딩 모델 로드
    embedding_model = SentenceTransformer(embedding_model_name)

    # 쿼리 임베딩
    if use_instruct_prefix: # Instruct 모델을 사용하는 경우 'query: ' 접두사 추가
        query = f"query: {query}"

    query_vector = embedding_model.encode(query).tolist()

    # Qdrant Client 로드
    client = QdrantClient(host='localhost', port=6333)

    # Qdrant 검색 수행
    search_results = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=top_k,
        with_payload=True, # 검색된 포인트의 메타 데이터
        with_vectors=False # 검색된 포인트의 임베딩 벡터 (필요 X)
    )

    # 결과 출력
    if not search_results:
        print(f"ERROR: 검색 결과가 없습니다.")
        
        return

    
    for rank, result in enumerate(search_results.points):
        payload = result.payload

        chunk_text = payload.get('text', '텍스트 없음')
        source_title = payload.get('title', '제목 없음')
        source_press = payload.get('press', '출처 없음')

        print(f"[{rank+1}]: \nchunk_text: {chunk_text}\nsource_title: {source_title}\nsource_press: {source_press}")


#### 데이터셋 로드

In [3]:
# 데이터셋 로드
dataset_id = 'daekeun-ml/naver-news-summarization-ko'
dataset = load_dataset(dataset_id, split='test')
documents_df = dataset.to_pandas()

# Example
print(f"문서 수 : {len(documents_df)}")
print(f"문서 Columns : {documents_df.columns}")

문서 수 : 2740
문서 Columns : Index(['date', 'category', 'press', 'title', 'document', 'link', 'summary'], dtype='object')


#### CHUNKING -> EMBEDDING -> INDEXING
* Chunk Size, Chunk Overlap, Embedding Model, Collection Name 확인 필수!

In [6]:
# 변수 설정
CHUNK_PARAMS = [(100, 20)]

EMBEDDING_MODEL_NAME = "xlm-r-large-en-ko-nli-ststb"
USE_INSTRUCT_PREFIX = False

for CHUNK_SIZE, CHUNK_OVERLAP in CHUNK_PARAMS:
    COLLECTION_NAME = f'XLM_{CHUNK_SIZE}'

    # log
    print(f"{COLLECTION_NAME} 컬렉션 생성 시작")

    chunk_results = chunking(documents_df, CHUNK_SIZE, CHUNK_OVERLAP)

    # log
    print(f"청킹 완료")

    embedding_indexing(chunk_results, EMBEDDING_MODEL_NAME, COLLECTION_NAME, USE_INSTRUCT_PREFIX)
    

XLM_100 컬렉션 생성 시작
청킹 완료


Batches:   0%|          | 0/218 [00:00<?, ?it/s]

C:\Users\dlaek\AppData\Local\Temp\ipykernel_11920\1190063013.py:53: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


임베딩 및 인덱싱 정상적으로 실행 완료 됨!


In [5]:
# 변수 설정
CHUNK_PARAMS = [(100, 20)]

EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-large-instruct"
USE_INSTRUCT_PREFIX = True

for CHUNK_SIZE, CHUNK_OVERLAP in CHUNK_PARAMS:
    COLLECTION_NAME = f'ME5_{CHUNK_SIZE}'

    # log
    print(f"{COLLECTION_NAME} 컬렉션 생성 시작")

    chunk_results = chunking(documents_df, CHUNK_SIZE, CHUNK_OVERLAP)

    # log
    print(f"청킹 완료")

    embedding_indexing(chunk_results, EMBEDDING_MODEL_NAME, COLLECTION_NAME, USE_INSTRUCT_PREFIX)
    

ME5_100 컬렉션 생성 시작
청킹 완료


Batches:   0%|          | 0/218 [00:00<?, ?it/s]

C:\Users\dlaek\AppData\Local\Temp\ipykernel_11920\1190063013.py:53: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


임베딩 및 인덱싱 정상적으로 실행 완료 됨!


#### Retrieval: Query -> Top K개의 Chunk 반환

In [ ]:
EMBEDDING_MODEL_NAME = "xlm-r-large-en-ko-nli-ststb"
COLLECTION_NAME = 'XLM_500'
USE_INSTRUCT_PREFIX = True
QUERY = '차량용 복합기능형 졸음 방지 단말기의 구성 요소 중 운전자의 얼굴 영상을 촬영하는 장치는 무엇입니까?'
TOP_K = 3

retrieval(EMBEDDING_MODEL_NAME, COLLECTION_NAME, QUERY, TOP_K, USE_INSTRUCT_PREFIX)

[1]: 
chunk_text: 아이일, 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔으며 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것으로, 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다.
source_title: 아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원
source_press: 머니투데이 
[2]: 
chunk_text: 악사 AXA 손해보험이 차량 출고 시 장착된 단말기로 차량의 실시간 주행 정보는 물론 운전자의 평소 운전 습관이나 차량의 사고 이력까지 투명하게 확인하고 차량 출고 시 장착된 단말기를 이용해 차량의 실시간 주행 정보는 물론 운전자의 평소 운전 습관이나 차량의 사고 이력까지 투명하게 확인하는 AXA커넥티드카 안전운전 할인 자동차보험 특별 약관을 신설했다고 4일 밝혔다.
source_title: 악사손보 안전운전하면 보험료 깎아드려요
source_press: 전자신문 
[3]: 
chunk_text: 6스트 GIST은 레벨4 기술의 자율주행 자동차가 도로 위의 경찰 수신호나 지시봉을 인식하기위한 세계 최대 규모의 수신호 데이터 데이터베이스 DB 를 구축해 이를 통해 자동차가 교통 수신호를 인식하고 정지하는 시연에 성공했다고 6일 밝혔으며 이를 통해 구축된 경찰 수신호 도로주행 이미지 보행자 및 경찰관 추적용 이미지 등의 데이터베이스는 향후 레벨 4 기술 이상의 자율주행 차량에 필수 요소인 교통 수신호 인지의 토대를 마련할 것으로 기대되며 이번 연구를 통해 구축된 경찰 수신호 도로주행 이미지 보행자 및 경찰관 추적용 이미지 등의 데이터베이스는 향후 레벨 4 기술 이상의 자율주행 차량에 필수 요소인 교통 수신호 인지의 토대를 마련할 것으로 기대된다.
source_title: 자율주행차가 교통경찰 수신호에 즉각 반응… 지스